# Task-Agnostic Exploration

参考
* [Task-agnostic Exploration in Reinforcement Learning](https://arxiv.org/abs/2006.09497)

Reward free RLでは任意の報酬に対しての探索を考えていましたが、Task-agnostic Explorationでは$N$個のタスクだけ考えます。
また、報酬関数の情報は集めたデータの状態行動についてしかもらえません（なのでTask-agnosticの方が難しいです）。
このとき、サンプル効率の下界が$\tilde{O}\left(\log (N) H^2 S A / \varepsilon^2\right)$になることが知られています。
これはReward Freeと比較すると$S$が取れて$\log (N)$になった形なので、Reward Freeよりも圧倒的に効率が良いわけですね。

---

**直感：なんで効率良くなるのか？**

先日、「Task-agnosticの設定でもどんな報酬関数が来るかわからない。なぜこっちのほうが効率が良くなるのか？」と聞かれて答えに窮してしまいました。
理論的にはHigh-probability Boundを考えていて、かつUnion Boundの範囲が狭くなるだけなのですが、ここに直感的な説明を書いておきます。

数学の期末試験を１回受けるとします。
あなたは試験で100点が取りたいですが、そんなに勉強したくありません。どれくらい勉強したら良いでしょうか？

今までの授業の内容を復習して、それなりの対策を取るとします。
今までの授業の中でもめちゃ難しい内容が運良くでなければ、この勉強の仕方でもいけそうです。

一方で、数学の試験を100回連続で受けるとします。
この場合、多分めちゃ難しい内容は試験の内容に入っていそうですね。100点取るには猛烈に勉強しないといけません。

つまり、受ける試験の数（タスクの数）が少なければ、運良くそれなりの性能を出すことは可能です。

---

以下では面倒なので定数係数は全部$\square$でまとめます。

## Task-Agnostic RL

次の問題設定を考えます。

問題設定

* エピソードの数：$K$
* $k$回目の方策：$\pi^k=\left\{\pi_0^k, \ldots, \pi_{H-1}^k\right\}$
* ホライズン：$H$
* 行動：$a_h \sim \pi_h^k (s_h^k)$
* 遷移確率：$s_{h+1}^k \sim P^\star_h\left(\cdot \mid s_h^k, a_h^k\right)$
* 状態行動の訪問回数：$N_h^k(s, a)=\sum_{i=0}^{k-1} \mathbf{1}\left\{\left(s_h^i, a_h^i\right)=(s, a)\right\}, \forall h, s, a$.

今回は次のBest-policy Identification問題を考えます

---

**Best-Policy Identification**

正の定数$\delta \in (0, 1)$と$\varepsilon \in (0, H)$を固定する．
アルゴリズムが任意のMDPに対して以下の条件を満たす非負整数値確率変数$\tau$を持つ場合，$\tau$の期待値を性能の指標とする：アルゴリズムは確率$1$で有限の$\tau$エピソード後に停止して方策$\pi$を出力し，確率$1-\delta$で
$$
    V_1^*(s_1) - V_1^\pi(s_1)
    \leq
    \varepsilon\,.
$$

---

### UCBZeroの説明

UCBZeroは[UCB-H](RL_UCB_H_regret_proof.ipynb)の無報酬への拡張とみなせます。

まずは探索時のアルゴリズムです。これで環境のデータを集めます。

\begin{equation}
\begin{aligned}
&\text { PARAMETERS: No. of tasks } N, \iota=\log (S A H K / p), b_t=c \sqrt{H^3(\log (N)+\iota) / t}, \alpha_t=\frac{H+1}{H+t} \text {. }\\
& \text { 1: }\text { initialize } \bar{Q}_h(s, a) \leftarrow H, N_h(s, a) \leftarrow 1 \text { for all }(s, a, h) \in S \times A \times H \\
& \text { 2: }\text { for episode } k=1, \ldots, K \text { do } \\
& \text { 3: }\quad \text { receive } s_1^k \text {. } \\
& \text { 4: }\quad \text { for step } h=1, \ldots, H \text { do } \\
& \text { 5: }\quad \quad \text { Take action } a_h^k \leftarrow \arg \max _a \bar{Q}_h\left(s_h^k, a\right), \text { and observe } s_{h+1}^k \cdot \\
& \text { 6: }\quad \quad t=N_h\left(s_h^k, a_h^k\right) \leftarrow N_h\left(s_h^k, a_h^k\right)+1 . \bar{V}_{h+1}\left(s_{h+1}^k\right)=\min \left(H, \max _a \bar{Q}_{h+1}\left(s_{h+1}^k, a\right)\right) \text {. } \\
& \text { 7: }\quad \quad \bar{Q}_h\left(s_h^k, a_h^k\right) \leftarrow\left(1-\alpha_t\right) \bar{Q}_h\left(s_h^k, a_h^k\right)+\alpha_t\left[\bar{V}_{h+1}\left(s_{h+1}^k\right)+2 b_t\right] \\
& \text { 8: }\quad \text { Return } \mathcal{D}=\left\{\left(s_h^k, a_h^k\right)\right\}_{h \in[H], k \in[K]}
\end{aligned}
\end{equation}

続いて、集めたデータで方策を最適化します。今回のTask-agnostic RLではReward Freeと違って、集めた箇所の報酬関数の情報しかもらえません。

\begin{equation}
\begin{aligned}
& \text { 1: } \text { initialize } \Pi=\emptyset, Q_h(s, a) \leftarrow H, N_h(s, a) \leftarrow 0 \text { for all }(s, a, h) \in S \times A \times H \text {. } \\
& \text { 2: for episode } k=1, \ldots, K \mathbf{d o} \\
& \text { 3: } \quad \Pi \text {.append }\left(\pi_k\right) \text {, where } \pi_k \text { is the greedy policy w.r.t. the current }\left\{Q_h\right\}_{h \in[H]} . \\
& \text { 4: } \quad \text { for step } h=1, \ldots, H \text { do } \\
& \text { 5: } \quad \quad t=N_h\left(s_h^k, a_h^k\right) \leftarrow N_h\left(s_h^k, a_h^k\right)+1, V_{h+1}\left(s_{h+1}^k\right)=\min \left(H, \max _a Q_{h+1}\left(s_{h+1}^k, a\right)\right) . \\
& \text { 6: } \quad \quad Q_h\left(s_h^k, a_h^k\right) \leftarrow\left(1-\alpha_t\right) Q_h\left(s_h^k, a_h^k\right)+\alpha_t\left[r_h^k+V_{h+1}\left(s_{h+1}^k\right)+b_t\right] . \\
& \text { 7: Return the (non-stationary) stochastic policy equivalent to uniformly sampling from } \Pi .
\end{aligned}
\end{equation}

このアルゴリズムはどちらもモデルフリーであることに注意しましょう。そのためメモリ効率が良いです。
また、探索時には$\bar{Q}$, 最適化時には$Q$をQ関数に使っていることに注意しましょう。

このリグレットの導出は[UCB-H](RL_UCB_H_regret_proof.ipynb)とほぼ同じですが、一つ注意するべき点があります。

まず、UCB-Hと最適化は同じなので、次の(a)が[UCB-H](RL_UCB_H_regret_proof.ipynb)の２より成立します。

$$
\left(V_1^{*(n)}-V_1^{\pi_k(n)}\right)\left(s_1\right) \stackrel{\text { (a) }}{\leq}\left(V_h^{k(n)}-V_h^{\pi_k(n)}\right)\left(s_1\right)
\leq \max _a\left(Q_h^{k(n)}-Q_h^{\pi_k(n)}\right)\left(s_1, a\right)
$$

$Q$の質はコントロールできないので、コントロール可能な$\bar{Q}$を使ってバウンドします。

#### １. $\left(Q_h^{k(n)}-Q_h^{\pi_k(n)}\right)(s, a) \leq\left(\bar{Q}_h^k-\bar{Q}_h^{\bar{\pi}_k}\right)(s, a)$を証明しよう

再帰的に示していきます。まず、丁寧に分解すれば

$$
\begin{aligned}
& {\left[\left(\bar{Q}_h^k-\bar{Q}_h^{\bar{\pi}_k}\right)-\left(Q_h^k-Q_h^{\pi_k}\right)\right](s, a) } \\
= & \alpha_t^0\left(Q_h^{\pi_k}-\bar{Q}_h^{\bar{\pi}_k}\right)(s, a) \\
& \stackrel{\text { (1) }}{+}\sum_{i=1}^t \alpha_t^i\left[\left(\bar{V}_{h+1}^{k_i}-\bar{V}_{h+1}^{\bar{\pi}_k}\right)-\left(V_{h+1}^{k_i}-V_{h+1}^{\pi_k}\right)\right]\left(s_{h+1}^{k_i}\right) \\
& \stackrel{\text { (2) }}{+}\sum_{i=1}^t \alpha_t^i\left[\left[\hat{\mathbb{P}}^{k_i}-\mathbb{P}_h\right]\left[\bar{V}_{h+1}^{\pi_k}-V_{h+1}^{\pi_k}\right](s, a)\right] \\
& \stackrel{\text { (3) }}{+}\sum_{i=1}^t \alpha_t^i b_i .
\end{aligned}
$$

$h+1$で$\left(Q_{h+1}^{k(n)}-Q_{h+1}^{\pi_k(n)}\right)(s, a) \leq\left(\bar{Q}_{h+1}^k-\bar{Q}_{h+1}^{\bar{\pi}_k}\right)(s, a)$が成り立っているとすれば、(1)は再帰的にバウンド可能です。

(2)は[UCB-H](RL_UCB_H_regret_proof.ipynb)でやった遷移確率のバウンドを考えれば簡単です。Azuma-Hoeffdingでいけます。

(3)は明らかに正です。

よって再帰的に$\left(Q_h^{k(n)}-Q_h^{\pi_k(n)}\right)(s, a) \leq\left(\bar{Q}_h^k-\bar{Q}_h^{\bar{\pi}_k}\right)(s, a)$が成立します。

#### 2. リグレットのバウンド

ここまで来たら後は簡単です。
$$
\begin{aligned}
\left(V_1^{*(n)}-V_1^{\pi_k(n)}\right)\left(s_1\right) & \stackrel{\text { (a) }}{\leq}\left(V_h^{k(n)}-V_h^{\pi_k(n)}\right)\left(s_1\right) \leq \max _a\left(Q_h^{k(n)}-Q_h^{\pi_k(n)}\right)\left(s_1, a\right) \\
& \stackrel{(b)}{\leq} \max _a\left(\bar{Q}_h^k-\bar{Q}_h^{\pi_k}\right)\left(s_1, a\right) \stackrel{(c)}{=}\left(\bar{V}_h^k-\bar{V}_h^{\bar{\pi}_k}\right)\left(s_1\right)
\end{aligned}
$$

(a)は上でみたやつ、(b)は１．を使う、(c)は無報酬のせいで成立します。
最後に$\bar{Q}$は報酬が０のときのUCB-Hなので、

$$
\sum_{k=1}^K\left(V_1^{*(n)}-V_1^{\pi_k(n)}\right)\left(s_1\right) \leq \sum_{k=1}^K\left(\bar{V}_1^k-\bar{V}_1^{\bar{\pi}_k}\right)\left(s_1\right) \leq \sqrt{(\log N+\iota) H^5 S A K}
$$

を得ます。